In [1]:
import sys
sys.path.append('..')

import yaml
import os
import requests

In [4]:
with open(f'../configs/main.yaml', 'r') as file:
    config = yaml.safe_load(file)
data_config = config['path']['data']
root_dir = data_config['root']
rawdata_dir = f"{root_dir}/{data_config['raw']}/"
rawdata_dir

'/home/sachinks/Data/raw/'

In [6]:
import os
import requests
from urllib.parse import urljoin, urlparse

def download_directory(url, local_path):
    # Create the local directory if it doesn't exist
    os.makedirs(local_path, exist_ok=True)

    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the base URL for relative links
        base_url = urlparse(url)

        # Parse the HTML content of the page
        page_content = response.text

        # Use a regular expression to find all anchor tags
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(page_content, 'html.parser')
        links = soup.find_all('a')

        # Iterate through the links and download files and subdirectories
        for link in links:
            href = link.get('href')
            link_url = urljoin(base_url.geturl(), href)

            # Create the local path by joining the local_path and the filename
            local_file_path = os.path.join(local_path, os.path.basename(href))

            # Check if the link points to a file or a subdirectory
            if '.' in os.path.basename(href):
                # Download the file
                with open(local_file_path, 'wb') as file:
                    file.write(requests.get(link_url).content)
            else:
                # Recursively download subdirectories
                download_directory(link_url, local_file_path)
    else:
        print(f"Failed to retrieve {url} - Status Code {response.status_code}")

# Specify the URL of the directory you want to download and the local directory where you want to save it
source_url = "https://labshare.cshl.edu/shares/library/repository/38599/2pData/"
local_directory = rawdata_dir

download_directory(source_url, local_directory)


ModuleNotFoundError: No module named 'bs4'